In [ ]:
import torch
from torch import nn
from torchvision import models

### Sample Data (NCHW)

In [ ]:
sample = torch.rand([8, 3, 32, 32])
sample224 = torch.rand([8, 3, 224, 224])

### Testing

In [ ]:
model = models.efficientnet_v2_s()
model

In [ ]:
import torchvision
from functools import partial
SE_layer = torchvision.ops.SqueezeExcitation(32, 2, partial(nn.SiLU, True))
SE_layer

In [ ]:
import numpy as np
np.random.beta(1, 1, 30)

### Handcrafts of Modifying Models from torchvision.models

In [ ]:
# resnext
model = models.resnext50_32x4d()
#print(model)

net = nn.Sequential(
    model.conv1,
    model.bn1,
    model.relu,
    nn.Sequential(),#model.maxpool,
    model.layer1,
    model.layer2,
    model.layer3,
    model.layer4,
)
#print(net(sample).shape)

model.maxpool = nn.Sequential()
model.fc = nn.Linear(in_features=2048, out_features=100, bias=True)
#print(model)

In [ ]:
# efficientnet_v2
model = models.efficientnet_v2_s()
#print(model)

net = nn.Sequential(
    model.features[0], # 3, 24, downsample (wrong proper noun)
    model.features[1], # 24, 24
    model.features[2], # 24, 48, downsample (wrong proper noun)
    model.features[3], # 48, 64, downsample (wrong proper noun)
    model.features[4], # 64, 128, downsample (wrong proper noun)
    model.features[5], # 128, 160
    model.features[6], # 160, 256, downsample (wrong proper noun)
    model.features[7], # 256, 1280
)
#print(net(sample).shape)

model.classifier[1] = nn.Linear(in_features=1280, out_features=100, bias=True)
#print(model)

### Original Version of Cifar_ResNet (20 layers)

In [ ]:
class BasicBlock(torch.nn.Module):
    def __init__(self, inplanes: int, planes: int, down: bool = False) -> None:
        super().__init__()
        self.down = down
        self.conv1 = (
            torch.nn.Conv2d(inplanes, planes, 3, stride=2, padding=1, bias=False) if down
            else torch.nn.Conv2d(inplanes, planes, 3, padding='same', bias=False)
        )
        self.bn1 = torch.nn.BatchNorm2d(planes)
        self.conv2 = torch.nn.Conv2d(planes, planes, 3, padding='same', bias=False)
        self.bn2 = torch.nn.BatchNorm2d(planes)
        self.relu = torch.nn.ReLU(inplace=True)
        if self.down:
            self.downsample = torch.nn.Sequential(
                torch.nn.Conv2d(inplanes, planes, 1, stride=2, bias=False),
                torch.nn.BatchNorm2d(planes)
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.down:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class BottleNeck(torch.nn.Module):
    # unused, not confirm the correctness yet
    def __init__(self, inplanes: int, planes: int, outplanes: int, down: bool = False) -> None:
        super().__init__()
        self.down = down
        self.conv1 = torch.nn.Conv2d(inplanes, planes, 1, bias=False)
        self.bn1 = torch.nn.BatchNorm2d(planes)
        self.conv2 = (
            torch.nn.Conv2d(planes, planes, 3, stride=2, padding=1, bias=False) if down
            else torch.nn.Conv2d(planes, planes, 3, padding='same', bias=False)
        )
        self.bn2 = torch.nn.BatchNorm2d(planes)
        self.conv3 = torch.nn.Conv2d(planes, outplanes, 1, bias=False)
        self.bn3 = torch.nn.BatchNorm2d(outplanes)
        self.relu = torch.nn.ReLU(inplace=True)
        if self.down:
            self.downsample = torch.nn.Sequential(
                torch.nn.Conv2d(inplanes, outplanes, 1, stride=2, bias=False),
                torch.nn.BatchNorm2d(outplanes)
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.down:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class CIFAR_ResNet(torch.nn.Module):
    def __init__(self, num_classes: int = 100) -> None:
        super().__init__()
        self.inplanes = 16
        self.planes = self.inplanes
        self.stem = torch.nn.Sequential(
            torch.nn.Conv2d(3, self.inplanes, kernel_size=3, padding='same', bias=False),
            torch.nn.BatchNorm2d(self.inplanes),
            torch.nn.ReLU(inplace=True)
        )
        self.layer1 = torch.nn.Sequential(
            BasicBlock(self.inplanes, self.planes),
            BasicBlock(self.planes, self.planes),
            BasicBlock(self.planes, self.planes),
        )
        self.inplanes = self.planes
        self.planes *= 2
        self.layer2 = torch.nn.Sequential(
            BasicBlock(self.inplanes, self.planes, down=True),
            BasicBlock(self.planes, self.planes),
            BasicBlock(self.planes, self.planes),
        )
        self.inplanes = self.planes
        self.planes *= 2
        self.layer3 = torch.nn.Sequential(
            BasicBlock(self.inplanes, self.planes, down=True),
            BasicBlock(self.planes, self.planes),
            BasicBlock(self.planes, self.planes),
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(1),
            torch.nn.Flatten(),
            torch.nn.Dropout(p=0.2, inplace=True),
            torch.nn.Linear(self.planes, num_classes)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.classifier(x)
        return x

class ResNet50(torch.nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        # not complete yet
        pass

In [ ]:
model = CIFAR_ResNet()
#print(model)
#print(model(sample).shape)

### Lazy Version of Cifar_ResNet (20 layers)

In [ ]:
class LazyBasicBlock(torch.nn.Module):
    def __init__(self, planes: int, down: bool = False) -> None:
        super().__init__()
        self.down = down
        self.conv1 = (
            torch.nn.LazyConv2d(planes, 3, stride=2, padding=1, bias=False) if down
            else torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        )
        self.bn1 = torch.nn.LazyBatchNorm2d()
        self.conv2 = torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        self.bn2 = torch.nn.LazyBatchNorm2d()
        self.relu = torch.nn.ReLU(inplace=True)
        if self.down:
            self.downsample = torch.nn.Sequential(
                torch.nn.LazyConv2d(planes, 1, stride=2, bias=False),
                torch.nn.LazyBatchNorm2d()
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.down:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class LazyBottleNeck(torch.nn.Module):
    def __init__(self, planes: int, outplanes: int, down: bool = False) -> None:
        super().__init__()
        self.down = down
        self.conv1 = torch.nn.LazyConv2d(planes, 1, bias=False)
        self.bn1 = torch.nn.LazyBatchNorm2d()
        self.conv2 = (
            torch.nn.LazyConv2d(planes, 3, stride=2, padding=1, bias=False) if down
            else torch.nn.LazyConv2d(planes, 3, padding='same', bias=False)
        )
        self.bn2 = torch.nn.LazyBatchNorm2d()
        self.conv3 = torch.nn.LazyConv2d(outplanes, 1, bias=False)
        self.bn3 = torch.nn.LazyBatchNorm2d()
        self.relu = torch.nn.ReLU(inplace=True)
        if self.down:
            self.downsample = torch.nn.Sequential(
                torch.nn.LazyConv2d(outplanes, 1, stride=2, bias=False),
                torch.nn.LazyBatchNorm2d()
            )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        if self.down:
            identity = self.downsample(x)
        out += identity
        out = self.relu(out)
        return out

class Lazy_CIFAR_ResNet(torch.nn.Module):
    def __init__(self, num_classes: int = 100) -> None:
        super().__init__()
        self.planes = 16
        self.stem = torch.nn.Sequential(
            torch.nn.LazyConv2d(self.planes, kernel_size=3, padding='same', bias=False),
            torch.nn.LazyBatchNorm2d(),
            torch.nn.ReLU(inplace=True)
        )
        self.layer1 = torch.nn.Sequential(
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
        )
        self.planes *= 2
        self.layer2 = torch.nn.Sequential(
            LazyBasicBlock(self.planes, down=True),
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
        )
        self.planes *= 2
        self.layer3 = torch.nn.Sequential(
            LazyBasicBlock(self.planes, down=True),
            LazyBasicBlock(self.planes),
            LazyBasicBlock(self.planes),
        )
        self.classifier = torch.nn.Sequential(
            torch.nn.AdaptiveAvgPool2d(1),
            torch.nn.Flatten(),
            torch.nn.Dropout(p=0.2, inplace=True),
            torch.nn.LazyLinear(num_classes)
        )
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.stem(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.classifier(x)
        return x

class Lazy_ResNet50(torch.nn.Module):
    def __init__(self, num_classes: int = 1000) -> None:
        # not complete yet
        pass

In [ ]:
model = Lazy_CIFAR_ResNet()
#print(model)
#print(model(sample).shape)